# Eniac — the e-commerce

In [62]:
import os
import pandas as pd 

In [41]:
brands_cl = pd.read_csv('./Datasets/Clean/brands.csv')
orderlines_cl = pd.read_csv('./Datasets/Clean/orderlines.csv')
orders_cl = pd.read_csv('./Datasets/Clean/orders.csv')
products_cl = pd.read_csv('./Datasets/Clean/products.csv')

## Products

### How many products are not in the product table

In [42]:
products_cl

,Unnamed: 0,sku,name,desc,in_stock,type,price
0,0,RAI0007,Silver Rain Design mStand Support,Aluminum support compatible with all MacBook,1,8696,59.99
1,1,APP0023,Apple Mac Keyboard Keypad Spanish,USB ultrathin keyboard Apple Mac Spanish.,0,13855401,59.00
2,2,APP0025,Mighty Mouse Apple Mouse for Mac,mouse Apple USB cable.,0,1387,59.00
3,3,APP0072,Apple Dock to USB Cable iPhone and iPod white,IPhone dock and USB Cable Apple iPod.,0,1230,25.00
4,4,KIN0007,Mac Memory Kingston 2GB 667MHz DDR2 SO-DIMM,2GB RAM Mac mini and iMac (2006/07) MacBook Pr...,1,1364,34.99
...,...,...,...,...,...,...,...
7751,7802,MMW0012,"My MW Case MacBook Pro 13 ""(Late 2016) Blue Sa...",Avoid shock and damage to your MacBook Pro 13-...,0,13835403,29.99
7752,7803,MMW0013,"My MW Case MacBook Pro 13 ""(Late 2016) White S...",Avoid shock and damage to your MacBook Pro 13-...,1,13835403,29.99
7753,7804,MMW0016,"My MW Case MacBook Pro 13 ""(Late 2016) Black",Avoid shock and damage to your MacBook Pro 13-...,1,13835403,29.99
7754,7805,MMW0015,"My MW Case MacBook Pro 13 ""(Late 2016) Gray",Avoid shock and damage to your MacBook Pro 13-...,1,13835403,29.99


In [43]:
orderlines_cl.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 293982 entries, 0 to 293981
Data columns (total 7 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   Unnamed: 0        293982 non-null  int64  
 1   id                293982 non-null  int64  
 2   id_order          293982 non-null  int64  
 3   product_quantity  293982 non-null  int64  
 4   sku               293982 non-null  object 
 5   unit_price        293982 non-null  float64
 6   date              293982 non-null  object 
dtypes: float64(1), int64(4), object(2)
memory usage: 15.7+ MB


In [44]:
products_cl.dropna(subset=['price'], inplace=True)

orderlines_cl_dropped_sku_set = set(orderlines_cl.sku)  # all SKUs which are present in orderlines
products_cl_sku_set = set(products_cl.sku)              # all SKU which are present in products

# len(orderlines_cl_dropped_sku_set), len(products_cl) #Output: (6048, 378395)

# check these masks for products which are not exict in product table
orderlines_cl_dropped_sku_mask = ~orderlines_cl.sku.isin(products_cl_sku_set)
print(len(orderlines_cl_dropped_sku_mask))

orderlines_cl_dropped_sku_mask = orderlines_cl.loc[orderlines_cl_dropped_sku_mask, :]

# drop these products
orderlines_cl_dropped = orderlines_cl.drop(orderlines_cl_dropped_sku_mask.index, axis=0)
print(len(orderlines_cl_dropped))

# products_compl = products_cl.copy()
products_cl.isna().sum()


293982
290756


Unnamed: 0    0
sku           0
name          0
desc          0
in_stock      0
type          0
price         0
dtype: int64

## Orders and Orderlines

### Completed orders vs Other states

In [45]:
orders_cl_compl = orders_cl.loc[orders_cl['state'] == 'Completed']

orders_cl_exclud_compl = orders_cl.loc[orders_cl['state'] != 'Completed']
orders_cl_compl.shape, orders_cl_exclud_compl.shape, orders_cl.shape

((46605, 5), (180299, 5), (226904, 5))

In [46]:
print('****ORDERS**** ', orders_cl_compl.order_id.nunique(), orders_cl_compl.shape[0])
print('****ORDERS-LINES**** ', orderlines_cl_dropped.id_order.nunique(), orderlines_cl_dropped.shape[0])

****ORDERS****  46605 46605
****ORDERS-LINES****  202924 290756


### How many unique orders do we have in orders and how do these numbers correlate with orderliness

In [47]:
orderlines_cl_dropped

,Unnamed: 0,id,id_order,product_quantity,sku,unit_price,date
0,0,1119109,299539,1,OTT0133,18.99,2017-01-01 00:07:19
1,1,1119110,299540,1,LGE0043,399.00,2017-01-01 00:19:45
2,2,1119111,299541,1,PAR0071,474.05,2017-01-01 00:20:57
3,3,1119112,299542,1,WDT0315,68.39,2017-01-01 00:51:40
4,4,1119113,299543,1,JBL0104,23.74,2017-01-01 01:06:38
...,...,...,...,...,...,...,...
293977,293978,1650199,527398,1,JBL0122,42.99,2018-03-14 13:57:25
293978,293979,1650200,527399,1,PAC0653,141.58,2018-03-14 13:57:34
293979,293980,1650201,527400,2,APP0698,9.99,2018-03-14 13:57:41
293980,293981,1650202,527388,1,BEZ0204,19.99,2018-03-14 13:58:01


In [48]:
orders_cl_compl

,Unnamed: 0,order_id,created_date,total_paid,state
1,1,241423,2017-11-06 13:10:02,136.15,Completed
2,2,242832,2017-12-31 17:40:03,15.76,Completed
3,3,243330,2017-02-16 10:59:38,84.98,Completed
5,5,245275,2017-06-28 11:35:37,149.00,Completed
6,6,245595,2017-01-21 12:52:47,112.97,Completed
...,...,...,...,...,...
226544,226549,527042,2018-03-14 11:47:50,18.98,Completed
226572,226577,527070,2018-03-14 11:50:48,24.97,Completed
226576,226581,527074,2018-03-14 11:51:42,24.97,Completed
226598,226603,527096,2018-03-14 11:58:40,34.96,Completed


In [49]:
# Unique IDs from the orders tabel
orders_cl_compl_id_set = set(orders_cl_compl.order_id)
print(len(orders_cl_compl_id_set)) # Output: 46605

# Make the set of the same numbers in ordder-lines table
orderlines_cl_same_id = orderlines_cl_dropped.id_order.isin(orders_cl_compl_id_set)
print(orderlines_cl_same_id.sum())  #Output: 61399

# invert our condition to finde all ids which are not exist in order table
orders_to_drop_ordlns_mask = orderlines_cl_dropped.loc[~orderlines_cl_same_id, :]

# drop all these rows
orderlines_cl_dropped = orderlines_cl_dropped.drop(orders_to_drop_ordlns_mask.index, axis=0 )

orderlines_compl = orderlines_cl_dropped.copy()

orderlines_compl

46605
61399


,Unnamed: 0,id,id_order,product_quantity,sku,unit_price,date
7,7,1119116,299545,1,OWC0100,47.49,2017-01-01 01:46:16
8,8,1119119,299546,1,IOT0014,18.99,2017-01-01 01:50:34
9,9,1119120,295347,1,APP0700,72.19,2017-01-01 01:54:11
11,11,1119126,299549,1,PAC0929,2565.99,2017-01-01 02:07:42
18,18,1119134,299556,1,CRU0039-A,60.90,2017-01-01 02:20:14
...,...,...,...,...,...,...,...
293598,293599,1649474,525664,1,TUC0207,16.52,2018-03-14 11:45:05
293615,293616,1649512,527070,2,APP0698,9.99,2018-03-14 11:49:01
293620,293621,1649522,527074,2,APP0698,9.99,2018-03-14 11:49:36
293643,293644,1649565,527096,3,APP0698,9.99,2018-03-14 11:54:35


In [53]:
# the same steps for orders
orderlines_cl_id_set = set(orderlines_compl.id_order)
orders_cl_compl_same_id = orders_cl_compl.order_id.isin(orderlines_cl_id_set)

print(len(orders_cl_compl_id_set), len(orders_cl_compl_same_id)) #Output: (46605, 46605)
orders_cl_compl_same_id_mask = orders_cl_compl.loc[~orders_cl_compl_same_id, :]
print(len(orders_cl_compl_same_id_mask)) #Output: 417
orders_cl_compl = orders_cl_compl.drop(orders_cl_compl_same_id_mask.index, axis=0)
orders_compl = orders_cl_compl.copy()

orders_compl

46605 46605
417


,Unnamed: 0,order_id,created_date,total_paid,state
1,1,241423,2017-11-06 13:10:02,136.15,Completed
2,2,242832,2017-12-31 17:40:03,15.76,Completed
3,3,243330,2017-02-16 10:59:38,84.98,Completed
5,5,245275,2017-06-28 11:35:37,149.00,Completed
6,6,245595,2017-01-21 12:52:47,112.97,Completed
...,...,...,...,...,...
226544,226549,527042,2018-03-14 11:47:50,18.98,Completed
226572,226577,527070,2018-03-14 11:50:48,24.97,Completed
226576,226581,527074,2018-03-14 11:51:42,24.97,Completed
226598,226603,527096,2018-03-14 11:58:40,34.96,Completed


In [54]:
print('****ORDERS**** ', orders_cl_compl.order_id.nunique(), orders_cl_compl.shape[0])
print('****ORDERS-LINES**** ', orderlines_cl_dropped.id_order.nunique(), orderlines_cl_dropped.shape[0])

****ORDERS****  46188 46188
****ORDERS-LINES****  46188 61399


## Revenue

### Calculation about revenue, discounts and outliers

* __Revenue__

In [55]:
orderlines_compl['revenue'] = orderlines_compl.product_quantity * orderlines_compl.unit_price
orderlines_compl

,Unnamed: 0,id,id_order,product_quantity,sku,unit_price,date,revenue
7,7,1119116,299545,1,OWC0100,47.49,2017-01-01 01:46:16,47.49
8,8,1119119,299546,1,IOT0014,18.99,2017-01-01 01:50:34,18.99
9,9,1119120,295347,1,APP0700,72.19,2017-01-01 01:54:11,72.19
11,11,1119126,299549,1,PAC0929,2565.99,2017-01-01 02:07:42,2565.99
18,18,1119134,299556,1,CRU0039-A,60.90,2017-01-01 02:20:14,60.90
...,...,...,...,...,...,...,...,...
293598,293599,1649474,525664,1,TUC0207,16.52,2018-03-14 11:45:05,16.52
293615,293616,1649512,527070,2,APP0698,9.99,2018-03-14 11:49:01,19.98
293620,293621,1649522,527074,2,APP0698,9.99,2018-03-14 11:49:36,19.98
293643,293644,1649565,527096,3,APP0698,9.99,2018-03-14 11:54:35,29.97


Total Revenue (12 month)

In [ ]:
print(round(orderlines_compl.loc[orderlines_compl.date.between("2017-01-01 00:00:00", "2017-12-31 23:59:59")]["revenue"].sum(), 2))

np.float64(12101316.05)

Avg Revenue (1 month)

In [58]:
print(round(orderlines_compl.loc[orderlines_compl.date.between("2017-01-01 00:00:00", "2017-12-31 23:59:59")]["revenue"].sum()/12,2))

1008443.0


Avg Revenue by order

In [59]:
orderlines_compl.loc[orderlines_compl.date.between("2017-01-01 00:00:00", "2017-12-31 23:59:59")].groupby(["id_order"]).agg({"revenue":"sum"}).mean()

revenue    333.903097
dtype: float64

In [61]:
orders_cl_compl = orders_cl_compl.rename(columns = {"Unnamed: 0": "index"})
orderlines_compl = orderlines_compl.rename(columns = {"Unnamed: 0": "index"})
orders_cl_compl.set_index("index")
orderlines_compl.set_index("index")

,id,id_order,product_quantity,sku,unit_price,date,revenue
index,,,,,,,
7,1119116,299545,1,OWC0100,47.49,2017-01-01 01:46:16,47.49
8,1119119,299546,1,IOT0014,18.99,2017-01-01 01:50:34,18.99
9,1119120,295347,1,APP0700,72.19,2017-01-01 01:54:11,72.19
11,1119126,299549,1,PAC0929,2565.99,2017-01-01 02:07:42,2565.99
18,1119134,299556,1,CRU0039-A,60.90,2017-01-01 02:20:14,60.90
...,...,...,...,...,...,...,...
293599,1649474,525664,1,TUC0207,16.52,2018-03-14 11:45:05,16.52
293616,1649512,527070,2,APP0698,9.99,2018-03-14 11:49:01,19.98
293621,1649522,527074,2,APP0698,9.99,2018-03-14 11:49:36,19.98


In [64]:
local_path = "./Datasets/Qual/"

if not os.path.exists(local_path):
    os.makedirs(local_path)
    
brands_cl.to_csv("./Datasets/Qual/brands.csv")
orderlines_compl.to_csv("./Datasets/Qual/orderlines.csv")
orders_cl_compl.to_csv("./Datasets/Qual/orders.csv")
products_cl.to_csv("./Datasets/Qual/products.csv")